In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import guidance
from textwrap import dedent
import json 

from grammar_guide import guide

In [2]:
model_name_or_path = "HuggingFaceTB/SmolLM-135M"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [12]:
res = guide(
    model,
    tokenizer,
    lark_grammar_filepath="../grammars/json.lark",
    seed_str="""{"name":""",
    prompt=dedent("""
    Here is a really long, nested JSON that extracts fields from this sentence:\n\nMy name is Joseph Smith, and I work at Apple. I'm 32 years old, and my interests include kayaking, skiing, snowboarding, and woodworking.\n\n```json\n
    """),
    draft_model=guidance.models.Transformers(
        model_name_or_path, echo=False
    ),
    token_healing=True,
    max_grammar_corrections=10,
    max_new_tokens=10,
    temperature=0.3,
)

Made a single_candidate correction...
Made a single_candidate correction...
Made a single_candidate correction...
Made a single_candidate correction...
Made a single_candidate correction...
Made a single_candidate correction...
Made a single_candidate correction...
Made a single_candidate correction...


Here is a really long , nested JSON that extracts fields from this sentence : 
 
 My name is Joseph Smith , and I work at Apple . I 'm 3 2 years old , and my interests include kay aking , skiing , snow boarding , and woodworking . 
 
 ``` json 

 {" name ": ":" Joseph Smith , 3 2 years old , and " " 
 " my interests include kay aking , skiing , 
 , ," age ": 3 2 , 
 " location ":" Apple " "} 
 
 ``` 
 
 I 'm trying to do 
 ," age ": 3 2 , 
 " location ":" Apple " , " ," job ":" K ay aking , skiing , snow " " 
 
 ``` 
 
 I 'm using the JSON "," job ":" K ay aking , skiing , snow " 

 , ," job _ type ":" Wood working "} 
 
 ``` 
 
 ," job _ type ":" Wood working " , " ," job _ type _ name ":" K ay aking " " 
 
 ``` 
 
 I want to extract the

In [13]:
print(json.dumps(json.loads(res.response),indent=4))

{
    "name": "Joseph Smith, 32 years old, and",
    "age": 32,
    "location": "Apple",
    "job": "Kayaking, skiing, snow",
    "job_type": "Woodworking",
    "job_type_name": "Kayaking"
}
